# Model Registry 🏬

Till now we've learn to run ML experiments and store those models locally but we still haven't yet decided which models are ready for production and if they're ready for production then how to keep track of all of them ? 

<img src="https://c.tenor.com/eYVYAKpC5jUAAAAM/packing-tryingtopacklike.gif" alt="packing bag" width="400">

Here comes the **Model Registry**  
>A model registry is a repository used to store and version trained machine learning (ML) models. Model registries greatly simplify the task of tracking models as they move through the ML lifecycle, from training to production deployments and ultimately retirement

![model_registry_banner](./images/model_registry_banner.jpg)

In **MLFlow** Model registry, we have labels assigned to models such as 
* staging : ready for production
* production : currently in production
* archieve : previously used production model (stored in case we need to rollback)

# Model Registry in MLFlow 

In [1]:
%time
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [2]:
%time
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
import xgboost as xgb

## Data Prep

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Train some models for demonstration

In [18]:
for alpha in [0.1, 0.4, 0.7] : 
    with mlflow.start_run():
        mlflow.set_tag("tutorial", "registry")
        
        mlflow.log_param("alpha" , alpha)
        lr = Lasso(alpha)
        lr.fit(X_train, y_train)

        y_pred = lr.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
        mlflow.sklearn.log_model(lr, artifact_path="mlflow_model")

In [21]:
with mlflow.start_run() :
    mlflow.set_tag("tutorial", "registry")
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10,
    )
    
    # tracking model using log_model
    mlflow.xgboost.log_model(booster, artifact_path="mlflow_model")
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)

[21:39:23] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583


Open MLflow UI, find our latest runs with the models and sort runs w.r.t metric `rmse`
![demo_models](./images/demo_models.jpg)

After this as a data scientist you need to **Analyze** these models and select a subset of them which can be used in production  
i.e. checking `size` of the model, training `duration` (more complex, more duration), `metric`, etc.  

For this experiment, I am going to select the first `xgboost` model and the third `Lasso` model cuz of the lesser `duration` and `rmse`

# Promoting models to registry

Go to our `xgboost` model present in the `Artifacts`section of the first run, 
![xgb_register](./images/xgb_register.jpg)

Create new model registry (if none is created before), 
![xgb_register_2.jpg](./images/xgb_register_2.jpg)

Similarly register that third `Lasso` model

Now if we wanna have a look at our register model, go to the `Models` section at the top of MLFlow UI
![registry_ui](./images/registry_ui.jpg)

If we click on `nyc-taxi-models` registry, we will be able to see the models which we had previously registered
![registry_ui2.jpg](./images/registry_ui2.jpg)


Lets see whats there inside each model,
![registry_ui3.jpg](./images/registry_ui3.jpg)

Ooof wait, there's nothing here ?
but PATHIK you had earlier said that we will be having information about its `hyperparameters`, `metrics`, etc. ?

YES dumbo, just click on the `Source Run` and you're good to go XD!

![source_run.jpg](./images/source_run.jpg)

anyways if you noticed, we now have a nice `model lineage`
>A model's lineage is a set of associations between a model and all the components that were involved in the creation of that mode

## Model Staging

We've just registered our models in registry but not yet declared which model is for **Production**, which is for **Staging** so lets do it 

Go to any of the registered model, lets say `Version 2`

![staging1](./images/staging1.jpg)

Simply Click on `Transistion to Staging` i.e. we are saying this model `Version 2` is listed in `Staging`

Do the same for other model and you should see something like this : 

![staging2.jpg](./images/staging2.jpg)

Well done data scientist ! 

![pew pew gif](https://c.tenor.com/FMFNIABbtgkAAAAC/finger-guns-michael-scott.gif)

Now we hand over the next task to lets say a deployment engineer to decide which model should be promoted to next stage

# MLFlow Client

If we don't want MLFlow UI then we can access everything in UI using `MlflowClient` 

In [22]:
from mlflow.tracking import MlflowClient

In [24]:
MLFLOW_TRACKING_URI= "sqlite:///mlflow.db"
client = MlflowClient(MLFLOW_TRACKING_URI)

In [33]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

## Explore runs in experiment_id=1

In [31]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1', # Experiment name in UI
    filter_string='tags.tutorial = "registry"', #  search filter which works like SQL queries 
    run_view_type=ViewType.ACTIVE_ONLY, 
    max_results=5, # results to return
    order_by=["metrics.rmse ASC"] # sort metrics in ascending
)

In [34]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ccbcf1ee02804891862b01150ccf5d56, rmse: 10.4758
run id: 08394232080c4587a32ac0d3a9fe36f7, rmse: 12.1432
run id: 59195799b10744208831eebbedabb61c, rmse: 12.2126
run id: 4220940ad45c44e8885228bcb5671da1, rmse: 12.2126


## Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

* Register a new version for the experiment `nyc-taxi-regressor`
* Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version 4 was created.
* Transition the version 4 to "Staging" and adding annotations to it.

In [35]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

### Registering model

In [37]:
run_id = "08394232080c4587a32ac0d3a9fe36f7" # 2nd model from above output
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-models")

Registered model 'nyc-taxi-models' already exists. Creating a new version of this model...
2022/06/20 00:49:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-models, version 3
Created version '3' of model 'nyc-taxi-models'.


<ModelVersion: creation_timestamp=1655666383559, current_stage='None', description=None, last_updated_timestamp=1655666383559, name='nyc-taxi-models', run_id='08394232080c4587a32ac0d3a9fe36f7', run_link=None, source='./mlruns/1/08394232080c4587a32ac0d3a9fe36f7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

If you check the MLflow UI for model registery then you will see registered model

![registered_model](./images/registered_model.jpg)

(btw I added version into `Production` before doing this)

## lets promote the model to staging

In [40]:
model_name = "nyc-taxi-models"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 3, stage: None


In [41]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1655666383559, current_stage='Staging', description=None, last_updated_timestamp=1655666848072, name='nyc-taxi-models', run_id='08394232080c4587a32ac0d3a9fe36f7', run_link=None, source='./mlruns/1/08394232080c4587a32ac0d3a9fe36f7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

Taddaa !
![staging3.jpg](./images/staging3.jpg)

That's pretty much it :)